In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import math
from tqdm import tqdm
import json
import os
from copy import deepcopy

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

/home/cheongwoong/miniconda3/envs/factual_knowledge_probing/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = '../data/analogy'
valid_path = os.path.join(data_path, 'valid.json')
test_path = os.path.join(data_path, 'test.json')
valid_data = json.load(open(valid_path, 'r'))
test_data = json.load(open(test_path, 'r'))

In [4]:
model_names = ['EleutherAI/gpt-neo-125m', 'EleutherAI/gpt-neo-1.3B', 'EleutherAI/gpt-neo-2.7B', 'EleutherAI/gpt-j-6b',
                 'meta-llama/Meta-Llama-3-8B', 'meta-llama/Meta-Llama-3-8B-Instruct',] 

for model_name in model_names:
    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).cuda()

    results = []

    for sample in tqdm(test_data):
        uid = sample['uid']
        sentence = sample['input']

        # Tokenize the input sentence
        tokens = tokenizer.tokenize(sentence)
        end_tail = len(tokens) - 2 # -2 when sentence == sample['input] and -1 when sentence == sample['input_without_period']
        tail = tokenizer.tokenize(sample['output'])
        start_tail = end_tail - len(tail) + 1

        if 'Llama' in model_name:
            start_tail += 1
            end_tail += 1

        inputs = tokenizer(sentence, return_tensors="pt").to('cuda')
        input_ids = inputs["input_ids"]

        # Get the model output and calculate loss
        with torch.no_grad():
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss
            perplexity = torch.exp(loss)  # Perplexity is exp(loss)

            tail_labels = deepcopy(input_ids)
            # tail_labels[:,start_tail:end_tail+1] = -100
            tail_labels[:,:start_tail] = -100
            tail_labels[:,end_tail+1:] = -100
            outputs_tail = model(input_ids, labels=tail_labels)
            loss_tail = outputs_tail.loss
            tail_ppl = torch.exp(loss_tail)

        result = {'uid': uid, 'ppl': perplexity.item(), 'tail_ppl': tail_ppl.item()}
        results.append(result)

    os.makedirs('results', exist_ok=True)
    output_model_name = model_name.split('/')[-1]
    with open(os.path.join('results', f'{output_model_name}.json'), 'w') as fout:
        json.dump(results, fout)

100%|██████████| 13656/13656 [28:00<00:00,  8.13it/s]
